In [8]:
import pandas as pd
import netCDF4 as nc
import numpy as np

In [53]:
asset_df = pd.read_csv("/home/aadrian/Documents/Clim_map/data/assets/assets.tsv", sep = "\t").dropna()
asset_df.head()

,City,Adress,Street,Prov,Zip,"Lat,Lon",Lat,Lon,Price
0,Brooks,220.0,2 St W,AB,T1R 0S3,"50.5665875158811, -111.89847582938071",50.566588,-111.898476,10000000.0
1,Pincher Creek,732.0,Main St,AB,T0K 1W0,"49.48547621120782, -113.94822344661871",49.485476,-113.948223,10000000.0
2,Cardston,204.0,Main St,AB,T0K 0K0,"49.200229091382745, -113.30140137225571",49.200229,-113.301401,10000000.0
3,High River,102.0,3 Ave SW,AB,T1V 1R3,"50.58033026999468, -113.87239511997352",50.580330,-113.872395,10000000.0
4,Calgary,339.0,8 Ave SW,AB,T2P 1C9,"51.04556519075384, -114.06998973306004",51.045565,-114.069990,10000000.0


In [54]:
fwi_df = pd.read_csv("/home/aadrian/Documents/Clim_map/bin/notebooks/fwi.csv", index_col=0)
sznlen_df.head()

,lon,lat,fwi
0,-144.750204,43.127567,--
1,-144.250611,43.127567,--
2,-143.751019,43.127567,--
3,-143.251426,43.127567,--
4,-142.751834,43.127567,--


In [ ]:
def get_closest_fwi(fwi_df, lat, lon):
    """Given a fwi_df of (lon, lat ) shape, and an input lon and lat, determine which lon and lat in the fwi df are closest to the input"""

    fwi_df['distance'] = np.sqrt((fwi_df['lat'] - lat) ** 2 + (fwi_df['lon'] - lon) ** 2)
    

    def extract_closest_nonmissing_fwi(fwi_df):
        fwi_df = fwi_df[fwi_df.fwi!='--']
        # Find the row with the minimum distance 
        closest_row = fwi_df.loc[fwi_df['distance'].idxmin()] 
        # Extract the closest latitude and longitude
        closest_lat = closest_row['lat']
        closest_lon = closest_row['lon']    

        closest_fwi = fwi_df.loc[(fwi_df['lat'] == closest_lat) & (fwi_df['lon'] == closest_lon)]
        return closest_fwi
    
    closest_fwi = extract_closest_nonmissing_fwi(fwi_df)
    closest_fwi = float(closest_fwi.fwi.values[0])
    return closest_fwi

lo_fwi = []
for i, row in asset_df.iterrows():
    fwi = get_closest_fwi(fwi_df=fwi_df, lat = row['Lat'], lon=row['Lon'])
    lo_fwi.append(fwi)


asset_df['FWI'] = lo_fwi


# Append Szn Len to asset df

In [63]:
sznlen_df = pd.read_csv("/home/aadrian/Documents/Clim_map/bin/notebooks/szn_len.csv", index_col=0)
sznlen_df

,lon,lat,szn_len
0,-144.750204,43.127567,NaN
1,-144.250611,43.127567,NaN
2,-143.751019,43.127567,NaN
3,-143.251426,43.127567,NaN
4,-142.751834,43.127567,NaN
...,...,...,...
12345,-52.325584,75.101490,NaN
12346,-51.825991,75.101490,NaN
12347,-51.326398,75.101490,NaN
12348,-50.826806,75.101490,NaN


In [78]:
def get_closest_sznlen(sznlen_df, lat, lon):
    """Given a sznlen_df of (lon, lat ) shape, and an input lon and lat, determine which lon and lat in the fznlen df are closest to the input"""

    sznlen_df['distance'] = np.sqrt((sznlen_df['lat'] - lat) ** 2 + (sznlen_df['lon'] - lon) ** 2)
    

    def extract_closest_nonmissing_fznlen(sznlen_df):
        sznlen_df = sznlen_df[~sznlen_df.szn_len.isna()]
        # Find the row with the minimum distance 
        closest_row = sznlen_df.loc[sznlen_df['distance'].idxmin()] 
        # Extract the closest latitude and longitude
        closest_lat = closest_row['lat']
        closest_lon = closest_row['lon']    

        closest_sznlen = sznlen_df.loc[(sznlen_df['lat'] == closest_lat) & (sznlen_df['lon'] == closest_lon)]
        return closest_sznlen
    
    closest_sznlen = extract_closest_nonmissing_fznlen(sznlen_df)
    closest_sznlen = float(closest_sznlen.szn_len.values[0])
    return closest_sznlen

lo_sznlen = []
for i, row in asset_df.iterrows():
    sznlen = get_closest_sznlen(sznlen_df=sznlen_df, lat = row['Lat'], lon=row['Lon'])
    lo_sznlen.append(sznlen)

asset_df['Season_Length'] = lo_sznlen
asset_df

,City,Adress,Street,Prov,Zip,"Lat,Lon",Lat,Lon,Price,FWI,Season_Length
0,Brooks,220.0,2 St W,AB,T1R 0S3,"50.5665875158811, -111.89847582938071",50.566588,-111.898476,10000000.0,47.891525,219.65735
1,Pincher Creek,732.0,Main St,AB,T0K 1W0,"49.48547621120782, -113.94822344661871",49.485476,-113.948223,10000000.0,53.704994,223.10602
2,Cardston,204.0,Main St,AB,T0K 0K0,"49.200229091382745, -113.30140137225571",49.200229,-113.301401,10000000.0,55.580292,222.25998
3,High River,102.0,3 Ave SW,AB,T1V 1R3,"50.58033026999468, -113.87239511997352",50.580330,-113.872395,10000000.0,49.142036,217.99535
4,Calgary,339.0,8 Ave SW,AB,T2P 1C9,"51.04556519075384, -114.06998973306004",51.045565,-114.069990,10000000.0,48.768559,215.99140
5,Calgary,411.0,58 Ave SE,AB,T2H 0P5,"51.00101526639408, -114.05628859100543",51.001015,-114.056289,10000000.0,48.768559,215.99140
6,Revelstoke,123.0,MacKenzie Ave,BC,V0E 2S0,"50.99901607946072, -118.19535130739008",50.999016,-118.195351,10000000.0,51.967316,215.96470
7,Vernon,2306.0,BC-6,BC,V1T 7E3,"50.25556096629964, -119.26298294789623",50.255561,-119.262983,10000000.0,54.976021,231.48398
8,Oliver,6306.0,Main St,BC,V0H 1T0,"49.18429295295311, -119.5506960294492",49.184293,-119.550696,10000000.0,59.842194,229.37207
9,Castlegar,1128.0,3 St,BC,V1N 1Z9,"49.326278247298106, -117.66505180249935",49.326278,-117.665052,10000000.0,57.604385,228.49263


In [79]:
asset_df.to_csv("../../data/assets/pro_assets.csv")